In [1]:
# 필요한 기본 패키지 준비

# 데이터 처리 필요 패키지
import numpy as np
import pandas as pd
import datetime as dt
import math

# 다중공선성(multicollinearity) 처리를 위한VIF 확인 패키지
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 시각화 필요 패키지
%matplotlib inline
from plotnine import *
import folium
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rc, font_manager
import seaborn as sns


# Machine Learning 분석 환경 준비

# 전처리, 스케일링
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

# 선형회귀분석
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from scipy import stats

# OLS회귀분석
import statsmodels.api as sm

# # GAM 일반화가법모형
# # LinearGAM, LogisticGAM, PoissonGAM, GammaGAM, InvGuss
# from pygam import LinearGAM, LogisticGAM, PoissonGAM, GammaGAM

# # Boosting

#데이터셋 분리
from sklearn.model_selection import train_test_split

# 한글 처리
font_name = font_manager.FontProperties(fname='C:/Windows/Fonts/NanumGothicCoding.ttf').get_name()
rc('font',family=font_name)

# - 마이너스 사인 처리
matplotlib.rcParams['axes.unicode_minus'] = False

# jupyter notebook에서 warning 무시하기
import warnings
warnings.filterwarnings("ignore")

In [2]:
def loadData():
    # 데이터 불러오기 (전처리 된 GS, 랄라블라, 날씨)
    gs = pd.read_csv('d:/project/contest/data/processed/p_gs.csv', parse_dates=['date'])
    lv = pd.read_csv('d:/project/contest/data/processed/p_lavla.csv', parse_dates=['date'])
    w = pd.read_csv('d:/project/contest/data/processed/p_wUVair_seoul.csv', parse_dates=['date'], index_col=0)
    sns_all = pd.read_csv('d:/project/contest/data/processed/social_all.csv', parse_dates=['date'])
    return gs, lv, w, sns_all
#########################################################################
# MAD 기반 예제코드
def mad_based_outlier(points, thresh=3.5):
    if len(points.shape) == 1:
        points = points[:,None]
    median = np.median(points, axis=0)
    diff = np.sum((points - median)**2, axis=-1)
    diff = np.sqrt(diff)
    med_abs_deviation = np.median(diff)
    modified_z_score = 0.6745 * diff / med_abs_deviation
    return modified_z_score > thresh 

# 출처: https://pythonanalysis.tistory.com/7 [Python 데이터 분석]
#########################################################################

# 소셜 데이터 처리를 위한 함수
# 1. 모든 소셜 데이터 column들의 첫번째는 : 날짜다.
# 2. 각 소셜데이터는 social_키워드.블로그/트위터/뉴스/총합 으로 되어 있다.
def changeColNames(d, before, after) : 
    # 컬럼이름 시리즈로 만들어 반환
    # 통합하기 쉽게, 모든 데이터들의 날짜컬럼 이름을 date로 통일
    new_col_names = ['date']
    new_col_names.extend(list(d.columns)[1:])
    d.columns = new_col_names
    return pd.Series(d.columns).apply(lambda x : x.replace(before,after))

#########################################################################
# modeling 함수로 만들어 처리하기
def linReg(df, item, cols_using):
    cols = cols_using
    X = df.loc[df['category']==item, cols]
    y = df.loc[df['category']==item,'qty']

    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=0)

    lin_model = LinearRegression().fit(X_train, y_train)
  
    print('LinearRegression을 이용한 %s의 회귀분석 결과 :'%item)
    print('훈련세트점수 : {:.2f}'.format(lin_model.score(X_train, y_train)))
    print('검증세트점수 : {:.2f}'.format(lin_model.score(X_test, y_test)))

    
def ridgeReg(df, item, cols_using):
    cols = cols_using
    X = df.loc[df['category']==item,cols]
    y = df.loc[df['category']==item,'qty']

    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=0)

    ridge = Ridge(alpha=0.1, normalize=True, random_state=0, tol=0.001).fit(X_train, y_train)
    
    print('RidgeRegression을 이용한 %s의 회귀분석 결과 :'%item)
    print('훈련세트점수 : {:.2f}'.format(ridge.score(X_train, y_train)))
    print('검증세트점수 : {:.2f}'.format(ridge.score(X_test, y_test)))


def lassoReg(df, item, cols_using):
    cols = cols_using
    X = df.loc[df['category']==item,cols]
    y = df.loc[df['category']==item,'qty']

    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=0)

    lasso = Lasso(alpha=0.1, max_iter=1000).fit(X=X_train, y=y_train)
  
    print('LassoRegression을 이용한 %s의 회귀분석 결과 :'%item)
    print('훈련세트점수 : {:.2f}'.format(lasso.score(X_train, y_train)) )
    print('검증세트점수 : {:.2f}'.format(lasso.score(X_test, y_test)) )

    #사용한 특성수
    print('사용한 특성수 : {}'.format(np.sum(lasso.coef_ != 0)) )
#########################################################################

# 자료가 1일 1행이라는 전제하에
# df길이를 이용하여 날짜수를 계산, 이후 2016년 1월1일을 1번째주 1일이라 기준하에
# 몇번째 주인지 알려주는 컬럼 추가
def addDayWeek(df):
    df_work = df.copy()
    df_work['day'] = pd.Series(range(1,df_work.shape[0]+1))
    df_work['week'] = df_work['day'].apply(lambda x : math.ceil(x/7))
    return df_work
#########################################################################
# 자료를 병합해주는 함수, 어떤 item인지 어느 컬럼을 기준으로 할지 받아서 병합
def mergeForAnalysis(df1, df2, df3, item, on_what='date'):
    merged_df = pd.merge(df1.loc[df1.category==item], df2, on=on_what, how='left')
    merged_df = pd.merge(merged_df, df3, on=on_what, how='left')
    return merged_df
#########################################################################
def lowVIF(df, n=7, cols_using =['temp', 'cloud', 'wind','humid', 'hpa', 'sun_time', 'lgt_time', 
       'SO2', 'CO', 'O3', 'NO2', 'PM10', 'PM25'] ):
    col_to_use = cols_using
    vif = pd.DataFrame()
    vif["VIF_Factor"] = [variance_inflation_factor(
        df[col_to_use].values, i) for i in range(df[col_to_use].shape[1])]
    vif["features"] = col_to_use
    vif.sort_values("VIF_Factor")
    lowest_vif = vif.sort_values("VIF_Factor")[:n].reset_index()
    lowest_vif.drop(columns='index', inplace=True)
    return lowest_vif
#########################################################################

def chiCheck_byP(df, col1, col2):
    chis = stats.chisquare(df[col1], df[col2])
    return chis

#########################################################################
# ols모델용 formula 생성
def formulaGen(target, ind_features):
    '''
    formulaGen(목표컬럼명,[변수컬럼명1, 변수컬럼명2,...])
    '''
    custom_formula = target + " ~ "
    for f in range(len(ind_features)):
        custom_formula += ind_features[f]
        if f!=(len(ind_features)-1):
            custom_formula += " + "
    return custom_formula


#########################################################################

In [3]:
def allAtOnce(sales_df, 
              weather_df, 
              item, 
              sales_cols=['date','bor_nm','gender','age_cd','category'],
              w_cols=['date','temp','humid','wind','rain','snow','cloud','sun_time','lgt_time','hpa',
                      'SO2','CO','O3','NO2','PM10','PM25'],
              n_vif_asc=7,
              target='qty'):
    '''
    sales_df = gs/lv데이터만 넣어주세요. 일단위로 종합된 데이터프레임에만 작동합니다.
    weather_df = w데이터만 넣어주세요. 일단위로 종합된 데이터프레임에만 작동합니다.
    item = str객체로, 한개의 물품만 넣어주세요
    w_cols = 기본적으로 'uv'데이터를 제외한 컬럼명들입니다.
    '''
    # 오류발생 어느정도 잡기 위한 부분
    if 'category' not in list(sales_df.columns):
        print('적법한 df를 넣으세요')
        return
    elif item not in list(sales_df.category.unique()):
        print('%s가 df안에 존재 하지 않습니다.'%item)
        return
    
    # GS/lv 서울시만
    sales_seoul = sales_df.loc[sales_df.pvn_nm =='서울특별시']
    w_seoul = weather_df.loc[weather_df['loc']==108]
    sales_seoul_grouped = sales_seoul[sales_cols+[target]].groupby(by=['date','category']).sum().reset_index()
    sales_seoul_grouped_w_item = pd.merge(sales_seoul_grouped.loc[sales_seoul_grouped.category==item], 
                                          w_seoul, on='date', how='left')

    Xy = sales_seoul_grouped_w_item[w_cols+['category',target]]
    
    low_vif_list = lowVIF(Xy,n_vif_asc)
    
    linReg(df=Xy,item=item,cols_using=low_vif_list.features)
    ridgeReg(df=Xy,item=item,cols_using=low_vif_list.features)
    lassoReg(df=Xy,item=item,cols_using=low_vif_list.features)

    cust_F = formulaGen(target=target,ind_features=low_vif_list.features)
    ols_model = sm.OLS.from_formula(cust_F, data=Xy)
    print(ols_model.fit().summary())

In [4]:
# 데이터 불러오기 (전처리 된 GS, 랄라블라, 날씨)
gs, lv, w, sns_all = loadData()
# gs = pd.read_csv('d:/project/contest/data/processed/p_gs.csv', parse_dates=['date'])
# lv = pd.read_csv('d:/project/contest/data/processed/p_lavla.csv', parse_dates=['date'])
# w = pd.read_csv('d:/project/contest/data/processed/p_wUVair_seoul.csv', parse_dates=['date'], index_col=0)
# sns_all = pd.read_csv('d:/project/contest/data/processed/social_all.csv', parse_dates=['date'])

In [5]:
allAtOnce(sales_df=gs, weather_df=w, item='아이스크림', n_vif_asc=4)

LinearRegression을 이용한 아이스크림의 회귀분석 결과 :
훈련세트점수 : 0.73
검증세트점수 : 0.71
RidgeRegression을 이용한 아이스크림의 회귀분석 결과 :
훈련세트점수 : 0.72
검증세트점수 : 0.71
LassoRegression을 이용한 아이스크림의 회귀분석 결과 :
훈련세트점수 : 0.73
검증세트점수 : 0.71
사용한 특성수 : 4
                            OLS Regression Results                            
Dep. Variable:                    qty   R-squared:                       0.723
Model:                            OLS   Adj. R-squared:                  0.722
Method:                 Least Squares   F-statistic:                     713.4
Date:                Mon, 15 Jul 2019   Prob (F-statistic):          1.29e-302
Time:                        18:34:19   Log-Likelihood:                -11172.
No. Observations:                1096   AIC:                         2.235e+04
Df Residuals:                    1091   BIC:                         2.238e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         